In [1]:
import datetime

import yfinance as yf 
import pandas as pd
import torch
import torch.nn as nn
from forexgym.utils import Query, format_datasets

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
end = datetime.datetime.now()
start = end - datetime.timedelta(days=1)

min_15 = yf.download('EURUSD=X', start=start, end=end, interval='15m').reset_index().drop(["Adj Close", "Volume"], axis=1)
h_1 = yf.download('EURUSD=X', start=start, end=end, interval='1h').reset_index().drop(["Adj Close", "Volume"], axis=1)

min_15["Datetime"] = pd.to_datetime(min_15["Datetime"], dayfirst=True, utc=True)
h_1["Datetime"] = pd.to_datetime(h_1["Datetime"], dayfirst=True, utc=True)

min_15.rename(columns={"Datetime": "Date"}, inplace=True)
h_1.rename(columns={"Datetime": "Date"}, inplace=True)

timeframes_dict = {
    "15m": min_15,
    "1H": h_1
}

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
h_1.tail()

,Date,Open,High,Low,Close
15,2024-10-14 14:00:00+00:00,1.091941,1.092180,1.090869,1.091703
16,2024-10-14 15:00:00+00:00,1.091822,1.091941,1.090869,1.090869
17,2024-10-14 16:00:00+00:00,1.090988,1.091107,1.090394,1.090394
18,2024-10-14 17:00:00+00:00,1.090394,1.090750,1.090037,1.090750
19,2024-10-14 18:00:00+00:00,1.090631,1.090869,1.090394,1.090513


In [5]:
def select_close(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    
    return pd.DataFrame(df["Close"])

def article_processor(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    df["x1"] = ((df["Close"] - df["Close"].shift(1) ) / df["Close"]).shift(1) 
    df["x2"] = ((df["High"] - df["High"].shift(1) ) / df["High"]).shift(1) 
    df["x3"] = ((df["Low"] - df["Low"].shift(1) ) / df["Low"]).shift(1) 
    df["x4"] = (df["High"] - df["Close"]) / df["Close"] 
    df["x5"] = (df["Close"] - df["Low"]) / df["Close"]
    
    return df.drop(["Date", "Open", "High", "Low", "Close"], axis=1) 

In [9]:
ticker = "EURUSD"
#timeframes = ["1m", "5m", "15m", "30m", "1H", "4H", "1D"]
timeframes = ["1H", "15m"]

query = Query(episode_length=256, trading_timeframe="15m", trading_column="Close")
# query.add_query(
#     timeframe="4H",
#     window_size=4,
#     data_processor=article_processor
# )
# query.add_query(
#     timeframe="1H",
#     window_size=4,
#     data_processor=article_processor
# )
query.add_query(
    timeframe="15m",
    window_size=16,
    data_processor=article_processor
)
data = format_datasets(query, timeframes_dict)

In [8]:
min_15.tail()

,Date,Open,High,Low,Close,x1,x2,x3,x4,x5
74,2024-10-14 18:30:00+00:00,1.090275,1.090513,1.090275,1.090513,-0.000327,-0.000109,-0.000327,0.000000,0.000218
75,2024-10-14 18:45:00+00:00,1.090513,1.090750,1.090513,1.090750,0.000327,0.000109,0.000218,0.000000,0.000218
76,2024-10-14 19:00:00+00:00,1.090631,1.090869,1.090631,1.090750,0.000218,0.000218,0.000218,0.000109,0.000109
77,2024-10-14 19:15:00+00:00,1.090750,1.090750,1.090513,1.090513,0.000000,0.000109,0.000109,0.000218,0.000000
78,2024-10-14 19:30:00+00:00,1.090513,1.090513,1.090394,1.090513,-0.000218,-0.000109,-0.000109,0.000000,0.000109


In [10]:
data.tail()

,Date,Trading_Price,15m_x1_0,15m_x2_0,15m_x3_0,15m_x4_0,15m_x5_0,15m_x1_1,15m_x2_1,15m_x3_1,...,15m_x1_14,15m_x2_14,15m_x3_14,15m_x4_14,15m_x5_14,15m_x1_15,15m_x2_15,15m_x3_15,15m_x4_15,15m_x5_15
57,2024-10-14 18:30:00+00:00,1.090513,-0.000327,-0.000109,-0.000327,0.000000,0.000218,0.000109,-0.000109,0.000000,...,0.000546,0.000873,0.000764,0.000874,0.000000,0.000546,0.000218,0.000327,0.000328,0.000437
58,2024-10-14 18:45:00+00:00,1.090750,0.000327,0.000109,0.000218,0.000000,0.000218,-0.000327,-0.000109,-0.000327,...,-0.001202,-0.000656,-0.000764,0.000328,0.000327,0.000546,0.000873,0.000764,0.000874,0.000000
59,2024-10-14 19:00:00+00:00,1.090750,0.000218,0.000218,0.000218,0.000109,0.000109,0.000327,0.000109,0.000218,...,0.000218,-0.000328,-0.000109,0.000109,0.000327,-0.001202,-0.000656,-0.000764,0.000328,0.000327
60,2024-10-14 19:15:00+00:00,1.090513,0.000000,0.000109,0.000109,0.000218,0.000000,0.000218,0.000218,0.000218,...,-0.000218,-0.000437,-0.000218,0.000000,0.000327,0.000218,-0.000328,-0.000109,0.000109,0.000327
61,2024-10-14 19:30:00+00:00,1.090513,-0.000218,-0.000109,-0.000109,0.000000,0.000109,0.000000,0.000109,0.000109,...,0.000436,0.000327,0.000436,0.000218,0.000109,-0.000218,-0.000437,-0.000218,0.000000,0.000327


In [32]:
data = data.drop(["Date", "Trading_Price"], axis=1)

In [13]:
class Network(nn.Module):
    def __init__(self, n_inputs: int, n_outputs: int, hidden_size: int = 128, n_layers: int = 2):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(n_inputs, hidden_size),
            *[
                nn.ReLU() if i % 2 == 0 else nn.Linear(hidden_size, hidden_size) for i in range(n_layers*2)
            ],
            nn.ReLU(),
            nn.Linear(hidden_size, n_outputs),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.softmax(self.layers(x))

In [14]:
model = torch.load("models/supervised_02.pt", weights_only=False)
model.eval()

Network(
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=2, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [34]:
features = torch.from_numpy(data.iloc[-3:].to_numpy()).to(device).to(dtype=torch.float32)


In [36]:
data.tail()

,1H_x1_0,1H_x2_0,1H_x3_0,1H_x4_0,1H_x5_0,1H_x1_1,1H_x2_1,1H_x3_1,1H_x4_1,1H_x5_1,...,15m_x1_14,15m_x2_14,15m_x3_14,15m_x4_14,15m_x5_14,15m_x1_15,15m_x2_15,15m_x3_15,15m_x4_15,15m_x5_15
28,-0.000437,-0.000656,-0.000437,0.000547,0.000109,0.000109,0.000547,0.000219,0.000766,0.000109,...,-0.000109,0.000000,0.000000,0.000109,0.000109,0.000000,-0.000109,0.000000,0.000000,0.000219
29,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,-0.000219,0.000000,-0.000109,0.000328,0.000000,-0.000109,0.000000,0.000000,0.000109,0.000109
30,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,0.000109,-0.000219,-0.000109,0.000000,0.000219,-0.000219,0.000000,-0.000109,0.000328,0.000000
31,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,-0.000109,0.000328,0.000109,0.000437,0.000000,0.000109,-0.000219,-0.000109,0.000000,0.000219
32,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,0.000328,0.000000,0.000000,0.000109,0.000328,-0.000109,0.000328,0.000109,0.000437,0.000000


In [37]:
model.forward(features)

tensor([[0.9932, 0.0068],
        [0.8571, 0.1429],
        [0.0063, 0.9937]], device='cuda:0', grad_fn=<SoftmaxBackward0>)